In [1]:
## Update sagemaker python sdk version
!pip install -U sagemaker

## Set data

copy data to s3

In [5]:
!ls ../playground/datamini

imgs  llava_mini.json


In [13]:
#cp data
! aws s3 cp ../playground/datamini/ s3://sagemaker-us-west-2-726335585155/yafei/data_llavasample/train/ --recursive --quiet

In [15]:
! aws s3 ls s3://sagemaker-us-west-2-726335585155/yafei/data_llavasample/train/

                           PRE .ipynb_checkpoints/
                           PRE imgs/
2024-05-14 03:43:02     393628 llava_mini.json


## Launch training

In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()
region = sess.boto_session.region_name

In [ ]:
import time
from sagemaker.estimator import Estimator

## pre-built docker in https://github.com/aws/deep-learning-containers/blob/master/available_images.md
image_uri = '763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker'

instance_count = 1
#instance_type = 'ml.p4d.24xlarge' ## p4d - 8*40G / p4de - 8*80G
instance_type = 'ml.g5.48xlarge' 

bucket=sagemaker.Session().default_bucket()
base_job_name="sagemaker-checkpoint-test"
checkpoint_in_bucket="checkpoints"
# The S3 URI to store the checkpoints
checkpoint_s3_bucket="s3://{}/{}/{}".format(bucket, base_job_name, checkpoint_in_bucket)


environment = {
    'NODE_NUMBER': str(instance_count),
    'MODEL_NAME_OR_PATH': 'liuhaotian/llava-v1.5-13b',
    #'OUTPUT_DIR': '/opt/ml/model',
    'OUTPUT_DIR': '/opt/ml/checkpoints'
}

estimator = Estimator(role=role,
                      entry_point='entry_single.py',
                      source_dir='./src',
                      base_job_name='a10-llava-train',
                      instance_count=instance_count,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      volume_size=512,
                      max_run=5*24*3600, #任务最大存续时间，默认2day，需要提交ticket提升quota最大28天 keep_alive_period_in_seconds=3600, #warmpool，为下一次训练保持机器&镜像（滚动续期，最大1hour）；需要开quota。
                      disable_profiler=True,
                      debugger_hook_config=False,
                      checkpoint_s3_uri=checkpoint_s3_bucket,
                      checkpoint_local_path='/opt/ml/checkpoints')


# # data in channel will be automatically copied to each node - /opt/ml/input/data/train1
# # should change data_path param to above path in torchrun
input_channel = {'train': 's3://sagemaker-us-west-2-726335585155/yafei/data_llavasample/train/'}
estimator.fit(input_channel)

# estimator.fit()

INFO:sagemaker:Creating training-job with name: a10-llava-train-2024-05-16-08-09-37-528


2024-05-16 08:09:40 Starting - Starting the training job...
2024-05-16 08:10:05 Pending - Training job waiting for capacity...
2024-05-16 08:10:37 Pending - Preparing the instances for training......
2024-05-16 08:11:39 Downloading - Downloading input data...
2024-05-16 08:12:01 Downloading - Downloading the training image.......